In [587]:
import numpy as np
import pandas as pd
import fitz
from bs4 import BeautifulSoup
import re
import pypandoc
from fuzzywuzzy import process

In [588]:
# songs_list = pd.DataFrame(columns=['track_id', 
#                                   'title', 
#                                   'artist', 
#                                   'artist2', 
#                                   'lang', 
#                                   'genre', 
#                                   'year', 
#                                   'album', 
#                                   'hp_status', 
#                                   'rs_score', 
#                                   'complex_score', 
#                                   'user_rating',
#                                   'hp_score',
#                                   'rank'])

# song_list.columns.head()
# song_list.to_csv('song_list.csv')
# song_list.to_csv('song_list.csv')
# song_list_test = pd.read_csv('~/song_list/songs_list.csv')
# song_list_test.head()

In [589]:


ratings_list = pd.DataFrame(columns=['track_id',
                               'hp_status', 
                                'rs_score', 
                                'complex_score', 
                                  'user_rating',
                                  'hp_score',
                                  'rank'])

ratings_list.head()
ratings_list.to_csv('ratings_list.csv')
ratings_list.to_csv('songs_list.csv')
ratings_list = pd.read_csv('ratings_list.csv')
ratings_list.head()

,Unnamed: 0,track_id,hp_status,rs_score,complex_score,user_rating,hp_score,rank


In [590]:
rolling_stones_list = "rs_list.pdf"
pdf = fitz.open(rolling_stones_list)

text = ""
for page_num in range(len(pdf)):
    page = pdf.load_page(page_num)
    text +=page.get_text("text")

lines = text.split("\n")
non_empty_lines = [line for line in lines if line.strip() != ""]
cleaned_text = "\n".join(non_empty_lines)


rs_rank = []
rs_artist = []
rs_song = []
rs_year = []

list_lines = cleaned_text.split("\n")
list_lines = list_lines[5:]

# Specify the file name
#file_name = "output.txt"

# Open the file in write mode and write the contents of list_lines
#with open(file_name, "w") as file:
#    for line in list_lines:
#        file.write(line + "\n")

# Specify the file name
file_name = "output.txt"

# Read the contents of the file back into list_lines
with open(file_name, "r") as file:
    list_lines = [line.strip() for line in file.readlines()]

        
idx = 1
for line in list_lines:
    if idx > 4:
        idx = 1
    if idx == 1:
        rs_rank.append(line)
        idx += 1
    elif idx == 2:
        rs_artist.append(line)
        idx += 1
    elif idx == 3:
        rs_song.append(line)
        idx += 1
    elif idx == 4:
        rs_year.append(line)
        idx = 1
    else:
        print("Error")

#print(rs_rank)
rank_cleaned = [re.sub("[^0-9]", "", line) for line in rs_rank]
# print(rank_cleaned)

rs_df = pd.DataFrame({'rs_rank': rank_cleaned, 'rs_artist': rs_artist, 'rs_song': rs_song, 'rs_year': rs_year})

rs_df.to_csv('rs_df.csv')

In [591]:
#NME

nme = "nme_list.rtf"
text = pypandoc.convert_file(nme, 'plain')
text = text.replace('\xa0', ' ')
text = text.replace('\n', ' ')

lines = text.split(";")


nme_list = []
for line in lines:
    line.strip()
    parts = line.split('–')
    if len(parts) == 2:
        rank_artist = parts[0].split('. ')
        rank = rank_artist[0].strip()
        artist = rank_artist[1]
        song = parts[1].strip()
        nme_list.append([rank, artist, song])
print(nme_list)

nme_df = pd.DataFrame(nme_list, columns=['nme_rank', 'nme_artist', 'nme_song'])
print(nme_df.head())

nme_df.to_csv('nme_df.csv')



[['1', 'Nirvana ', 'Smells Like Teen Spirit'], ['2', 'Joy Division ', 'Love Will Tear Us Apart'], ['3', 'Donna Summer ', 'I Feel Love'], ['4', 'The Smiths ', 'How Soon Is Now?'], ['5', 'The Strokes ', 'Last Night'], ['6', 'Pulp ', 'Common People'], ['7', 'Arctic Monkeys ', 'I Bet You Look Good on the Dancefloor'], ['8', 'New Order ', 'Blue Monday'], ['9', 'The Ronettes ', 'Be My Baby'], ['10', 'The Beach Boys ', 'Good Vibrations'], ['11', 'The Beatles ', 'Eleanor Rigby'], ['12', 'The Smiths ', 'There Is a Light That Never Goes Out'], ['13', 'Oasis ', 'Live Forever'], ['14', 'The Beach Boys ', 'God Only Knows'], ['15', 'David Bowie ', '“Heroes”'], ['16', 'The White Stripes ', 'Seven Nation Army'], ['17', 'The Beatles ', 'A Day in the Life'], ['18', 'Outkast ', 'Hey Ya!'], ['19', 'Blondie ', 'Heart of Glass'], ['20', 'Blur ', 'Girls & Boys'], ['21', 'The Specials ', 'Ghost Town'], ['22', "The La's ", 'There She Goes'], ['23', 'The Kinks ', 'Waterloo Sunset'], ['24', 'Bob Dylan ', 'Hurric

In [592]:
# Time 100

time = "time_list.csv"
time_df= pd.read_csv(time)
print(time_df.head())
time_df.to_csv('time_df.csv')




   Rank                                  Artist                      Song  \
0     1  Bing Crosby with the Ken Darby Singers           White Christmas   
1     2                         Michael Jackson               Billie Jean   
2     3                                   Queen         Bohemian Rhapsody   
3     4                                 Nirvana   Smells Like Teen Spirit   
4     5                             The Beatles  I Want to Hold Your Hand   

   Year  
0  1942  
1  1982  
2  1975  
3  1991  
4  1963  


In [593]:

df1 = rs_df
df2 = nme_df
df3 = time_df


def normalize(df, song_col, _artist_col):
    df[song_col] = df[song_col].str.lower()
    df[song_col] = df[song_col].str.strip()
    df[_artist_col] = df[_artist_col].str.lower()
    df[_artist_col] = df[_artist_col].str.strip()
    return df

df1 = normalize(df1, 'rs_song', 'rs_artist')
df2 = normalize(df2, 'nme_song', 'nme_artist')
df3 = normalize(df3, 'Song', 'Artist')

df3.rename(columns={'Year': 't_year', 'Song': 't_song', 'Artist': 't_artist', 'Rank' : 't_rank'}, inplace=True)
df1.head()

,rs_rank,rs_artist,rs_song,rs_year
0,1,bob dylan,like a rolling stone,1961
1,2,the rolling stones,satisfaction,1965
2,3,john lennon,imagine,1971
3,4,marvin gaye,what’s going on,1971
4,5,aretha franklin,respect,1967


In [594]:
# Initialize the DataFrame with the specified columns
df_match12 = pd.DataFrame(columns=[
    'rs_song', 'nme_song', 'rs_artist', 'nme_artist', 
    'rs_rank', 'nme_rank', 'rs_year'])

# Iterate over each row in df1
for index, row in df1.iterrows():
    song = row['rs_song']
    artist = row['rs_artist']
    
    # Find the best match for the song in df2
    match = process.extractOne(song, df2['nme_song'])
    
    if match:
        matched_idx = df2[df2['nme_song'] == match[0]].index[0]
        
        # Find the best match for the artist in the matched row of df2
        match_artist = process.extractOne(artist, [df2.loc[matched_idx, 'nme_artist']], score_cutoff=80)
        
        if match_artist and match[1] > 80:
            print(f"Match found: {song} ({artist}) -> {match[0]} ({match_artist[0]}) with scores {match[1]}, {match_artist[1]}")
            # Add a new row to df_match12 with the matched details
            df_match12.loc[len(df_match12)] = [
                song, match[0], artist, match_artist[0], 
                row['rs_rank'], df2.loc[matched_idx, 'nme_rank'], 
                row['rs_year']
            ]
        else:
            print(f"No match for artist: {song} ({artist}) -> {match[0]} with score {match[1]}")
    else:
        print(f"No match for song: {song} ({artist})")

# Display the first 100 rows of the combined DataFrame
df_match12.head(100)
df_match12.to_csv('df_match12.csv')
len(df_match12)

Match found: like a rolling stone (bob dylan) -> like a rolling stone (bob dylan) with scores 100, 100
No match for artist: satisfaction (the rolling stones) -> one with score 72
Match found: imagine (john lennon) -> imagine (john lennon) with scores 100, 100
Match found: what’s going on (marvin gaye) -> what's going on (marvin gaye) with scores 100, 100
Match found: respect (aretha franklin) -> respect (aretha franklin) with scores 100, 100
Match found: good vibrations (the beach boys) -> good vibrations (the beach boys) with scores 100, 100
No match for artist: johnny b. goode (chuck berry) -> one with score 60
Match found: hey jude (the beatles) -> hey jude (the beatles) with scores 100, 100
Match found: smells like teen spirit (nirvana) -> smells like teen spirit (nirvana) with scores 100, 100
No match for artist: what'd i say (part 1&2) (ray charles) -> i feel love with score 86
Match found: my generation (the who) -> my generation (the who) with scores 100, 100
Match found: a cha

144

In [595]:
# Initialize the DataFrame with the specified columns
df_match13 = pd.DataFrame(columns=[
    'rs_song', 't_song', 'rs_artist', 't_artist', 
    'rs_rank', 't_rank', 'rs_year', 't_year'
])

# Iterate over each row in df3
for index, row in df3.iterrows():
    song = row['t_song']
    artist = row['t_artist']
    
    # Find the best match for the song in df1
    match = process.extractOne(song, df1['rs_song'].dropna())
    
    if match:
        matched_idx = df1[df1['rs_song'] == match[0]].index[0]
        
        # See if the artist is a good match for the matched_idx
        match_artist = process.extractOne(artist, [df1.loc[matched_idx, 'rs_artist']], score_cutoff=80)
        
        if match_artist and match[1] > 80:
            print(f"Match found: {song} ({artist}) -> {match[0]} ({match_artist[0]}) with scores {match[1]}, {match_artist[1]}")
            # Add a new row to df_match13 with the matched details
            df_match13.loc[len(df_match13)] = [
                df1.loc[matched_idx, 'rs_song'], song, 
                df1.loc[matched_idx, 'rs_artist'], artist, 
                df1.loc[matched_idx, 'rs_rank'] if 'rs_rank' in df1.columns else None, 
                row['t_rank'] if 't_rank' in row else None, 
                df1.loc[matched_idx, 'rs_year'] if 'rs_year' in df1.columns else None, 
                row['t_year'] if 't_year' in row else None
            ]
        else:
            print(f"No match for artist: {song} ({artist}) -> {match[0]} with score {match[1]}")
    else:
        print(f"No match for song: {song} ({artist})")

# Display the first 100 rows of the combined DataFrame
df_match13.head(300)
df_match13.to_csv('df_match13.csv')

No match for artist: white christmas (bing crosby with the ken darby singers) -> white room with score 86
Match found: billie jean (michael jackson) -> billie jean (michael jackson) with scores 100, 100
Match found: bohemian rhapsody (queen) -> bohemian rhapsody (queen) with scores 100, 100
Match found: smells like teen spirit (nirvana) -> smells like teen spirit (nirvana) with scores 100, 100
Match found: i want to hold your hand (the beatles) -> i want zo hold your hand (the beatles) with scores 96, 100
Match found: i heard it through the grapevine (marvin gaye) -> i heard it through the grapevine (marvin gaye) with scores 100, 100
Match found: stayin’ alive (bee gees) -> stayin' alive (the bee gees) with scores 100, 90
No match for artist: over the rainbow (judy garland) -> rain with score 90
Match found: nothing compares 2 u (sinéad o’connor) -> nothing compares 2 u (sinead o'connor) with scores 100, 97
Match found: hey ya! (outkast) -> hey ya (outkast) with scores 100, 100
No matc

In [596]:
# Initialize the DataFrame with the specified columns
df_match23 = pd.DataFrame(columns=[
    'nme_song', 't_song',  'nme_artist', 't_artist', 
    'nme_rank', 't_rank'
])

# Iterate over each row in df3
for index, row in df3.iterrows():
    song = row['t_song']
    artist = row['t_artist']
    
    # Find the best match for the song in df1
    match = process.extractOne(song, df2['nme_song'].dropna())
    
    if match:
        matched_idx = df2[df2['nme_song'] == match[0]].index[0]
        
        # See if the artist is a good match for the matched_idx
        match_artist = process.extractOne(artist, [df2.loc[matched_idx, 'nme_artist']], score_cutoff=80)
        
        if match_artist and match[1] > 90:
            print(song, artist, match[0], match[1], match_artist[0], match_artist[1])
            # Add a new row to df_match23 with the matched details
            df_match23.loc[len(df_match23)] = [
                df2.loc[matched_idx, 'nme_song'], song,  # nme_song and t_song
                df2.loc[matched_idx, 'nme_artist'], artist,  # nme_artist and t_artist
                df2.loc[matched_idx, 'nme_rank'] if 'nme_rank' in df2.columns else None,  # nme_rank
                row['t_rank'] if 't_rank' in row else None  # t_rank
            ]
        else:
            print("No match", song, artist, match[0], match[1], match_artist[0] if match_artist else None, match_artist[1] if match_artist else None)
    else:
        print("No match for song", song, artist)

# Display the first 100 rows of the combined DataFrame
df_match23.head(100)
df_match23.to_csv('df_match23.csv')

No match white christmas bing crosby with the ken darby singers (white man) in hammersmith palais 86 the clash 86
billie jean michael jackson billie jean 100 michael jackson 100
No match bohemian rhapsody queen maps 68 None None
smells like teen spirit nirvana smells like teen spirit 100 nirvana 100
i want to hold your hand the beatles i want to hold your hand 100 the beatles 100
i heard it through the grapevine marvin gaye i heard it through the grapevine 100 marvin gaye 100
No match stayin’ alive bee gees stan 68 None None
No match over the rainbow judy garland i bet you look good on the dancefloor 86 None None
nothing compares 2 u sinéad o’connor nothing compares 2 u 100 sinead o' connor 93
hey ya! outkast hey ya! 100 outkast 100
No match cheek to cheek fred astaire & ginger rogers with leo reisman’s orchestra i want to hold your hand 86 None None
jailhouse rock elvis presley jailhouse rock 100 elvis presley 100
No match johnny b. goode chuck berry one 60 None None
No match st. loui

In [597]:
df_merge = pd.merge(df_match12, df_match13, how='outer', on='rs_song', suffixes=('', '13'))
df_merge.head()
df_merge.to_csv('df_merge_12_13.csv')

In [598]:
df_merge = pd.merge(df_merge, df_match23, how='outer', on='t_song', suffixes=('', '23'))
df_merge.head()
df_merge.to_csv('df_merge_12_13_23.csv')

In [599]:


# Assuming df_merge is already defined and merged
# Identify columns with suffixes
suffix = '23'
columns_to_combine = [col for col in df_merge.columns if col.endswith(suffix)]

# Combine suffixed columns by overwriting NaN values
for col in columns_to_combine:
    original_col = col.rstrip(suffix)
    if original_col in df_merge.columns:
        df_merge[original_col] = df_merge[original_col].combine_first(df_merge[col])
    else:
        df_merge[original_col] = df_merge[col]

# Drop the suffixed columns
df_merge.drop(columns=columns_to_combine, inplace=True)

# Display the first 100 rows of the cleaned DataFrame
df_merge.head(100)


# Assuming df_merge is already defined and merged
# Identify columns with suffixes
suffix = '13'
columns_to_combine = [col for col in df_merge.columns if col.endswith(suffix)]

# Combine suffixed columns by overwriting NaN values
for col in columns_to_combine:
    original_col = col.rstrip(suffix)
    if original_col in df_merge.columns:
        df_merge[original_col] = df_merge[original_col].combine_first(df_merge[col])
    else:
        df_merge[original_col] = df_merge[col]

# Drop the suffixed columns
df_merge.drop(columns=columns_to_combine, inplace=True)

# Display the first 100 rows of the cleaned DataFrame
df_merge.head(100)
df_merge.to_csv('df_merge_cleaned.csv')

In [600]:
for idx, row in df1.iterrows():
    song = row['rs_song']
    rs_artist = row['rs_artist']
    rs_rank = row['rs_rank']
    rs_year = row['rs_year']
    if song in df_merge['rs_song']:
        continue
    else: 
       df_merge.loc[len(df_merge)] = [song, None, rs_artist, None, rs_rank, rs_year, None, None, None, None, None]
df_merge.to_csv('df_merge_df1.csv')

/var/folders/kd/_n33wrhn6b323mv4n5w5t8v80000gn/T/ipykernel_50879/3975651299.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_merge.loc[len(df_merge)] = [song, None, rs_artist, None, rs_rank, rs_year, None, None, None, None, None]
/var/folders/kd/_n33wrhn6b323mv4n5w5t8v80000gn/T/ipykernel_50879/3975651299.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_merge.loc[len(df_merge)] = [song, None, rs_artist, None, rs_rank, rs_year, None, None, None, None, None]
/var/folders/kd/_n33wrhn6b32

In [601]:
for idx, row in df2.iterrows():
    song = row['nme_song']
    nme_artist = row['nme_artist']
    nme_rank = row['nme_rank']
    if song in df_merge['rs_song']:
        continue
    else: 
       df_merge.loc[len(df_merge)] = [None, song, None, nme_artist, None, nme_rank, None, None, None, None, None]
df_merge.to_csv('df_merge_df2.csv')

/var/folders/kd/_n33wrhn6b323mv4n5w5t8v80000gn/T/ipykernel_50879/2304743674.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_merge.loc[len(df_merge)] = [None, song, None, nme_artist, None, nme_rank, None, None, None, None, None]
/var/folders/kd/_n33wrhn6b323mv4n5w5t8v80000gn/T/ipykernel_50879/2304743674.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_merge.loc[len(df_merge)] = [None, song, None, nme_artist, None, nme_rank, None, None, None, None, None]
/var/folders/kd/_n33wrhn6b323m

In [602]:
for idx, row in df3.iterrows():
    song = row['t_song']
    t_artist = row['t_artist']
    t_rank = row['t_rank']
    t_year = row['t_year']
    if song in df_merge['t_song']:
        continue
    else: 
       df_merge.loc[len(df_merge)] = [None, None, None, None, None, None, None, song, t_artist, t_rank, t_year]
df_merge.to_csv('df_merge_df3.csv')

In [603]:
df_merge['t_rank'] = pd.to_numeric(df_merge['t_rank'], errors='ignore').astype('Int64')
df_merge['t_year'] = pd.to_numeric(df_merge['t_year'], errors='ignore').astype('Int64')
df_merge.head()

/var/folders/kd/_n33wrhn6b323mv4n5w5t8v80000gn/T/ipykernel_50879/210560473.py:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_merge['t_rank'] = pd.to_numeric(df_merge['t_rank'], errors='ignore').astype('Int64')
/var/folders/kd/_n33wrhn6b323mv4n5w5t8v80000gn/T/ipykernel_50879/210560473.py:2: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_merge['t_year'] = pd.to_numeric(df_merge['t_year'], errors='ignore').astype('Int64')


,rs_song,nme_song,rs_artist,nme_artist,rs_rank,nme_rank,rs_year,t_song,t_artist,t_rank,t_year
0,NaN,99 problems,NaN,jay-z,NaN,40,NaN,99 problems,jay-z,61,<NA>
1,NaN,all my friends,NaN,lcd soundsystem,NaN,91,NaN,all my friends,lcd soundsystem,90,<NA>
2,baba o'riley,NaN,the who,NaN,340,NaN,1971,baba o’riley,the who,41,1971
3,"shake, rattle and roll",NaN,big joe turner,NaN,126,NaN,1954,ball and chain,willie mae “big mama’ thornton,95,1968
4,be my baby,be my baby,the ronettes,the ronettes,22,9,1963,be my baby,the ronettes,18,1963


In [604]:
df_merge['artist'] = df_merge['rs_artist'].combine_first(df_merge['nme_artist']).combine_first(df_merge['t_artist'])
df_merge['song'] = df_merge['rs_song'].combine_first(df_merge['nme_song']).combine_first(df_merge['t_song'])
df_merge['year'] = df_merge['rs_year'].combine_first(df_merge['t_year'])
df_merge.drop(columns=['rs_artist', 'nme_artist', 't_artist', 'rs_song', 'nme_song', 't_song', 'rs_year', 't_year'], inplace=True)

df_merge.head()

,rs_rank,nme_rank,t_rank,artist,song,year
0,NaN,40,61,jay-z,99 problems,<NA>
1,NaN,91,90,lcd soundsystem,all my friends,<NA>
2,340,NaN,41,the who,baba o'riley,1971
3,126,NaN,95,big joe turner,"shake, rattle and roll",1954
4,22,9,18,the ronettes,be my baby,1963


In [605]:
df_merge['hp_status'] = 1
df_merge['user_rating'] = np.random.randint(1, 10, df_merge.shape[0])
df_merge['consolidated_score'] = None
df_merge.to_csv('df_merge_final.csv')

In [606]:
df_merge.head()

,rs_rank,nme_rank,t_rank,artist,song,year,hp_status,user_rating,consolidated_score
0,NaN,40,61,jay-z,99 problems,<NA>,1,3,None
1,NaN,91,90,lcd soundsystem,all my friends,<NA>,1,4,None
2,340,NaN,41,the who,baba o'riley,1971,1,4,None
3,126,NaN,95,big joe turner,"shake, rattle and roll",1954,1,6,None
4,22,9,18,the ronettes,be my baby,1963,1,9,None


In [607]:
def calculate_cumulative_score(rank_list, max_rank_list, weights, hp_status, user_rating):
    """
    Calculate the cumulative score for a song based on its ranks in multiple lists, hp_status, and user rating.

    Parameters:
    - rank_list: List of ranks for the song on each list. If the song doesn't appear on a list, use None or 0.
    - max_rank_list: List of the maximum rank (i.e., the total number of songs) for each list.
    - weights: List of weights corresponding to each list.
    - hp_status: Status indicating if the song has a special boost (1 for yes, 0 for no).
    - user_rating: User rating for the song on a scale of 0 to 10.

    Returns:
    - Final score on a 10-point scale.
    """
    normalized_scores = []
    
    for rank, max_rank in zip(rank_list, max_rank_list):
        if pd.isna(rank) or rank == 0:
            normalized_scores.append(0)  # No score if the song doesn't appear in the list
        else:
            normalized_score = 1 - (rank - 1) / (max_rank - 1)  # Normalize the ranking
            normalized_scores.append(normalized_score)
    
    # Calculate the weighted cumulative score
    weighted_score = sum([w * s for w, s in zip(weights, normalized_scores)]) / sum(weights)
    
    # Apply hp_status boost or reduction
    if hp_status == 1:
        weighted_score *= 1.1  # Boost by 10%
    elif hp_status == 0:
        weighted_score *= 0.9  # Reduce by 10%

    # Incorporate user rating strongly
    user_rating_weight = 1.5  # Strong weight for user rating
    final_score = (10 * weighted_score + user_rating * user_rating_weight) / (1 + user_rating_weight)
    
    return final_score

# Example inputs
rank_list = [1, 1, None]  # Song ranks: appears on list 1 and 2, not on list 3
max_rank_list = [500, 500, 100]  # Max ranks for each list
weights = [1, 1, 1]  # Weights for each list
hp_status = 1  # Special boost status
user_rating = 8  # User rating on a scale of 0 to 10


In [608]:

# Calculate the cumulative score
for idx, row in df_merge.iterrows():
    rank_list = [pd.to_numeric(row['rs_rank'], errors='coerce'), 
                 pd.to_numeric(row['nme_rank'], errors='coerce'), 
                 pd.to_numeric(row['t_rank'], errors='coerce')]
    max_rank_list = [500, 500, 100]
    weights = [1, 1, 1]
    hp_status = row['hp_status']
    user_rating = row['user_rating']
    df_merge.loc[idx, 'consolidated_score'] = calculate_cumulative_score(rank_list, max_rank_list, weights, hp_status, user_rating)
df_merge.head()

,rs_rank,nme_rank,t_rank,artist,song,year,hp_status,user_rating,consolidated_score
0,NaN,40,61,jay-z,99 problems,<NA>,1,3,3.729815
1,NaN,91,90,lcd soundsystem,all my friends,<NA>,1,4,3.750286
2,340,NaN,41,the who,baba o'riley,1971,1,4,3.744348
3,126,NaN,95,big joe turner,"shake, rattle and roll",1954,1,6,4.773339
4,22,9,18,the ronettes,be my baby,1963,1,9,9.462911
